In [1]:
import numpy as np
from sklearn import preprocessing
import os.path
import librosa
import warnings
import scipy.stats as stats
import scipy as sp
from sklearn.preprocessing import MinMaxScaler
from scipy import spatial

#### **2. Extracção de Features.**
2.1. Processar as features do ficheiro top100_features.csv.

2.1.1. Ler o ficheiro e criar um array numpy com as features disponibilizadas.

In [2]:
def extract_features():
    data = np.genfromtxt("Features/top100_features.csv", delimiter = ",", skip_header = 1)
    data = data[:, 1:-1]
    return data


2.1.2. Normalizar as features no intervalo [0, 1].

In [3]:
def normalize_features(data):
    max_values = data.max(axis = 0)
    min_values = data.min(axis = 0)
    return (data-min_values)/(max_values - min_values)

2.1.3. Criar e gravar em ficheiro um array numpy com as features extraídas (linhas = músicas; colunas = valores das features).

In [4]:
def save_to_file(path, data):
    np.savetxt(path, data, delimiter=",", fmt="%.6f")

2.2. Extrair features da framework librosa. <br>

2.2.1. Para os 900 ficheiros da BD, extrair as seguintes features (sugestão: guardar
todas as músicas na mesma pasta):
- Features Espectrais: mfcc, spectral centroid, spectral bandwidth, spectral
contrast, spectral flatness e spectral rolloff.
- Features Temporais: F0, rms e zero crossing rate.
- Outras features: tempo.
- Utilize os parâmetros por omissão do librosa (sr = 22050 Hz, mono, window
length = frame length = 92.88 ms e hop length = 23.22 ms).
- Guarde as features num array numpy 2D, com número de linhas = número de
músicas e número de colunas = número de features

In [5]:
def extract_music_features(music):
    path = "Music/"
    
    data, sampling_rate = librosa.load(path + music)

    mfcc = librosa.feature.mfcc(y=data, sr=sampling_rate, n_mfcc = 13)
    spectral_centroid = librosa.feature.spectral_centroid(y=data, sr=sampling_rate)
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=data, sr=sampling_rate)
    spectral_contrast = librosa.feature.spectral_contrast(y=data, sr=sampling_rate)
    spectral_flatness = librosa.feature.spectral_flatness(y=data)
    spectral_rolloff = librosa.feature.spectral_rolloff(y = data, sr = sampling_rate)
    f0 = librosa.yin(y=data, fmin=20, fmax=11025)
    f0[f0 == 11025] = 0
    rms = librosa.feature.rms(y=data)
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y=data)
    tempo = librosa.beat.tempo(y=data, sr=sampling_rate)
        
    return mfcc, spectral_centroid, spectral_bandwidth, spectral_contrast, spectral_flatness, spectral_rolloff, f0, rms, zero_crossing_rate, tempo

2.2.2. Calcular as 7 estatísticas típicas sobre as features anteriores: média, desvio
padrão, assimetria (skewness), curtose (kurtosis), mediana, máximo e mínimo.
Para o efeito, utilizar a biblioteca scipy.stats (e.g., scipy.stats.skew).

In [6]:
def calculate_statistics(feature, axis_):
    feat_mean = feature.mean(axis = axis_)
    feat_std = feature.std(axis = axis_)
    feat_skewness = stats.skew(feature, axis = axis_)
    feat_kurtosis = stats.kurtosis(feature, axis = axis_)
    feat_median = np.median(feature, axis = axis_)
    feat_max = feature.max(axis = axis_)
    feat_min = feature.min(axis = axis_)
    
    if(axis_ == 1):
        result = np.array([])
        for i in range(len(feat_mean)):
            result = np.hstack((result, feat_mean[i], feat_std[i], feat_skewness[i], feat_kurtosis[i], feat_median[i], feat_max[i], feat_min[i]))
        return result
    else:
        # f0
        return np.hstack(([feat_mean], [feat_std], [feat_skewness], [feat_kurtosis], [feat_median], [feat_max], [feat_min]))
    

#### **3.Implementação de métricas de similaridade.**

3.1. Desenvolver o código Python/numpy para calcular as seguintes métricas de similaridade:

3.1.1. Distância Euclidiana 

In [7]:
def euclidean_distance(a,b):
    distance = np.linalg.norm(a-b)
    return distance, distance

3.1.2. Manhattan distance

In [8]:
def manhattan_distance(a, b):
    distance = sp.spatial.distance.cityblock(a, b)
    return distance, distance

3.1.3. Cosine distance

In [9]:
def cosine_distance(a, b):
    distance = spatial.distance.cosine(a, b)
    return distance, distance


In [10]:
def calculate_distances(features, feature_type):
    data = np.empty((len(features), len(features)))
 
    for i in range (0, len(features)):
        for j in range (i, len(features)):
            if i == j:
                data[i][j] = 0
            elif feature_type == "Euclidean":
                data[i][j], data[j][i] = euclidean_distance(features[i], features[j])
            elif feature_type == "Manhattan":
                data[i][j], data[j][i] = manhattan_distance(features[i], features[j])
            elif feature_type == "Cosine":
                data[i][j], data[j][i] = cosine_distance(features[i], features[j])

    return data

In [11]:
def receive_distances(filename, data, type):
    if not os.path.exists(filename):
        data[data != data] = 0
        data_output = calculate_distances(data, type)
        save_to_file(filename, data_output)
        return data_output
    else:
        return np.genfromtxt(filename, delimiter = ",")   

In [12]:
def print_20_best_rankings(data, indexes):
    for i in range(1,21):
        print(f"\t{i}º - {data[indexes[i]]}")
    

#### **Main function**

In [13]:
if __name__ == "__main__" :
    warnings.filterwarnings("ignore")
    
    saved_features = "top100_features_normalized.csv"
    librosa_features = "librosa_features_normalized.csv"
    music_folder = os.listdir("Music")
    queries_folder = os.listdir("Queries")

    # Ex 2.1
    
    if os.path.exists(saved_features):
        data_normalized = np.genfromtxt(saved_features, delimiter = ",")
    else:
        data_extracted = extract_features()
        data_normalized = normalize_features(data_extracted)
        save_to_file(saved_features, data_normalized)
    
    
    # Ex 2.2
    if os.path.exists(librosa_features):
        librosa_data_normalized = np.genfromtxt(librosa_features, delimiter = ",")
    else:
        statistics = np.empty((0, 190))
        
        iteration = 1
        for music in music_folder:
            mfcc, spectral_centroid, spectral_bandwidth, spectral_contrast, spectral_flatness, spectral_rolloff, f0, rms, zero_crossing_rate, tempo = extract_music_features(music)
            
            mfcc_statistics = calculate_statistics(mfcc, 1)
            spectral_centroid_statistics = calculate_statistics(spectral_centroid, 1)
            spectral_bandwidth_statistics = calculate_statistics(spectral_bandwidth, 1)
            spectral_contrast_statistics = calculate_statistics(spectral_contrast, 1)
            spectral_flatness_statistics = calculate_statistics(spectral_flatness, 1)
            spectral_rolloff_statistics = calculate_statistics(spectral_rolloff, 1)
            f0_statistics = calculate_statistics(f0, 0)
            rms_statistics = calculate_statistics(rms, 1)
            zero_crossing_rate_statistics = calculate_statistics(zero_crossing_rate, 1)
            music_statistics = np.concatenate((mfcc_statistics.flatten(), 
                                                spectral_centroid_statistics,
                                                spectral_bandwidth_statistics,
                                                spectral_contrast_statistics.flatten(),
                                                spectral_flatness_statistics,
                                                spectral_rolloff_statistics,
                                                f0_statistics, 
                                                rms_statistics,
                                                zero_crossing_rate_statistics,
                                                tempo))
            statistics = np.vstack((statistics, music_statistics))
            iteration += 1 
            print(iteration)

        
        librosa_data_normalized = normalize_features(statistics)
        save_to_file(librosa_features, librosa_data_normalized)

    # Ex 3.1 e 3.2
    fich_names = ["data_euclidean_distance.csv", "librosa_euclidean_distance.csv","data_manhattan_distance.csv", "librosa_manhattan_distance.csv", "data_cosine_distance.csv", "librosa_cosine_distance.csv"]

    #       3.1.1
    data_euclidean_distance = receive_distances(fich_names[0], data_normalized, "Euclidean")
    librosa_euclidean_distance = receive_distances(fich_names[1], librosa_data_normalized, "Euclidean")
    
    #       3.1.2
    data_manhattan_distance = receive_distances(fich_names[2], data_normalized, "Manhattan")
    librosa_manhattan_distance = receive_distances(fich_names[3], librosa_data_normalized, "Manhattan")
    
    #       3.1.3
    data_cosine_distance = receive_distances(fich_names[4], data_normalized, "Cosine")
    librosa_cosine_distance = receive_distances(fich_names[5], librosa_data_normalized, "Cosine")


    # Ex 3.3
    for i in range(len(queries_folder)):

        print("\n--------------------- Song ", queries_folder[i])
        print("Ranking with data euclidean distance")
        indice = music_folder.index(queries_folder[i])

        indexes = np.argsort(data_euclidean_distance[indice])
        print_20_best_rankings(music_folder, indexes)

        print("\nRanking with librosa euclidean distance")
        indexes = np.argsort(librosa_euclidean_distance[indice])
        print_20_best_rankings(music_folder, indexes)
        
        print("\nRanking with data manhattan distance")
        indexes = np.argsort(data_manhattan_distance[indice])
        print_20_best_rankings(music_folder, indexes)
        
        print("\nRanking with librosa euclidean distance")
        indexes = np.argsort(librosa_manhattan_distance[indice])
        print_20_best_rankings(music_folder, indexes)
        
        print("\nRanking with data cosine distance")
        indexes = np.argsort(data_cosine_distance[indice])
        print_20_best_rankings(music_folder, indexes)
                
        print("\nRanking with librosa euclidean distance")
        indexes = np.argsort(librosa_cosine_distance[indice])
        print_20_best_rankings(music_folder, indexes)


--------------------- Song  MT0000202045.mp3
Ranking with data euclidean distance
	1º - MT0033841575.mp3
	2º - MT0027002641.mp3
	3º - MT0030487841.mp3
	4º - MT0008575372.mp3
	5º - MT0014576739.mp3
	6º - MT0030422114.mp3
	7º - MT0003390733.mp3
	8º - MT0027835071.mp3
	9º - MT0009188643.mp3
	10º - MT0010617945.mp3
	11º - MT0011145388.mp3
	12º - MT0009213083.mp3
	13º - MT0005331755.mp3
	14º - MT0002233402.mp3
	15º - MT0026727455.mp3
	16º - MT0005265641.mp3
	17º - MT0010344415.mp3
	18º - MT0000711493.mp3
	19º - MT0004428604.mp3
	20º - MT0018029465.mp3

Ranking with librosa euclidean distance
	1º - MT0005129157.mp3
	2º - MT0011899302.mp3
	3º - MT0012001409.mp3
	4º - MT0002233402.mp3
	5º - MT0007043504.mp3
	6º - MT0007799677.mp3
	7º - MT0004428604.mp3
	8º - MT0002161109.mp3
	9º - MT0011975274.mp3
	10º - MT0010624346.mp3
	11º - MT0009217411.mp3
	12º - MT0000092267.mp3
	13º - MT0007766156.mp3
	14º - MT0001676671.mp3
	15º - MT0005270263.mp3
	16º - MT0003787478.mp3
	17º - MT0011376343.mp3
	18º -